# NFL punting EPA analysis

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.interpolate import make_interp_spline, BSpline

sns.set(style="whitegrid", 
        palette="pastel", 
        font='Circular Spotify Text', 
        font_scale = 1.1, 
        rc= {'grid.color': '.95', 'xtick.color': '.3', 'ytick.color': '.3'})


## Data

In [2]:
df = pd.read_csv("../data/pbp_pre_2019.csv")

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (42,189,190,191,192,199,200,201,202,203,204,205,206,208,210,212,232,233,234,239,241,242) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,play_id,game_id,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,36,2019080151,ATL,DEN,DEN,away,ATL,ATL,35.0,2019-08-01,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
1,58,2019080151,ATL,DEN,DEN,away,ATL,DEN,74.0,2019-08-01,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,79,2019080151,ATL,DEN,DEN,away,ATL,DEN,71.0,2019-08-01,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,101,2019080151,ATL,DEN,DEN,away,ATL,DEN,71.0,2019-08-01,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,135,2019080151,ATL,DEN,DEN,away,ATL,DEN,71.0,2019-08-01,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [6]:
list(df.columns)

['play_id',
 'game_id',
 'home_team',
 'away_team',
 'posteam',
 'posteam_type',
 'defteam',
 'side_of_field',
 'yardline_100',
 'game_date',
 'quarter_seconds_remaining',
 'half_seconds_remaining',
 'game_seconds_remaining',
 'game_half',
 'quarter_end',
 'drive',
 'sp',
 'qtr',
 'down',
 'goal_to_go',
 'time',
 'yrdln',
 'ydstogo',
 'ydsnet',
 'desc',
 'play_type',
 'yards_gained',
 'shotgun',
 'no_huddle',
 'qb_dropback',
 'qb_kneel',
 'qb_spike',
 'qb_scramble',
 'pass_length',
 'pass_location',
 'air_yards',
 'yards_after_catch',
 'run_location',
 'run_gap',
 'field_goal_result',
 'kick_distance',
 'extra_point_result',
 'two_point_conv_result',
 'home_timeouts_remaining',
 'away_timeouts_remaining',
 'timeout',
 'timeout_team',
 'td_team',
 'posteam_timeouts_remaining',
 'defteam_timeouts_remaining',
 'total_home_score',
 'total_away_score',
 'posteam_score',
 'defteam_score',
 'score_differential',
 'posteam_score_post',
 'defteam_score_post',
 'score_differential_post',
 'no_sc

# Analysis!
## EPA per punt

In [4]:
df_punts = df[df['play_type'] == 'punt']

In [14]:
(df_punts.loc[:,['home_team', 'away_team', 'posteam', 'punter_player_name', 'punt_downed','epa']]
 .sort_values('epa', ascending=False)
 .head(20))

,home_team,away_team,posteam,punter_player_name,punt_downed,epa
1476,GB,HOU,GB,J.Scott,0.0,7.658086
2767,KC,CIN,KC,J.Fox,0.0,6.592964
2797,KC,CIN,KC,D.Colquitt,0.0,6.400412
5212,LA,DAL,DAL,C.Jones,0.0,6.260977
5199,LA,DAL,LA,J.Hekker,0.0,6.164452
6143,ATL,WAS,WAS,T.Way,0.0,5.833437
24,ATL,DEN,ATL,M.Bosher,0.0,5.807076
6164,ATL,WAS,WAS,T.Way,0.0,5.547033
6034,CIN,NYG,CIN,K.Huber,0.0,4.613073
4476,IND,CLE,CLE,J.Gillan,1.0,3.663105


In [19]:
df_punts.groupby(['punter_player_name'])['epa'].mean().reset_index().sort_values('epa', ascending=False)

,punter_player_name,epa
20,J.Hekker,1.061674
11,C.Jones,0.984893
14,D.Colquitt,0.955003
6,B.Miller,0.844480
18,J.Fox,0.680657
16,J.Bailey,0.648255
4,B.Colquitt,0.564220
42,S.Koch,0.555297
21,J.Scott,0.528241
15,I.Berryman,0.520455
